In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve

from catboost import Pool, CatBoostClassifier

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def read_and_preprocess(filepath):
    
    # initial reading
    data = pd.read_csv(filepath, index_col=0)
    
    # move species from index to the table
    data["species"] = data.index
    
    # create numeric idex
    data.index = range(len(data))
    
    # create categorical feature from Phylum (string)
    data["Phylum_Numeric"] = LabelEncoder().fit_transform(data["Phylum"].tolist())
    
    # remove features we will not use
    data = data.drop(["Phylum", "species", "occurrences", "NoSpecies", 
                      "C_Cnumeric", "SC_Numeric", "MaxD_Numeric", "System_Numeric"], axis=1)
    
    # create features and target dataframes
    features = data.drop(["extinct"], axis=1)
    target = data["extinct"]
    
    # create lists with categorical and continious features' names
    continious_cols = []
    categorical_cols = features.columns.drop(continious_cols).tolist()
    
    # make list of indexes
    categorical_idx = [features.columns.tolist().index(col) for col in categorical_cols]
        
    return features, target, categorical_idx

In [3]:
def model_me(filepath):
    
    # prepare data for modeling
    features, target, cat_idx = read_and_preprocess(filepath)
    
    # create cross-validation instance
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    
    # create holders for scores and feature importances
    cv_scores = []
    cv_roc_curves = []
    feature_importances = []
    indexes = {}
    
    # loop over different validation splits and save results
    for i, (train_idx, test_idx) in enumerate(cv.split(features, target)):
        
        # Gradient boosting model instance
        model = CatBoostClassifier(loss_function="Logloss", random_seed=0)
        
        # Create Pool data classes for train/test
        pool_train = Pool(features.iloc[train_idx, :], target[train_idx], cat_features=cat_idx)
        pool_test = Pool(features.iloc[test_idx, :], target[test_idx], cat_features=cat_idx)
        
        # Train model
        model.fit(pool_train, verbose=False)
        
        # save score from the individual split
        cv_scores.append(roc_auc_score(target[test_idx], model.predict_proba(pool_test)[:, 1]))
        
        # save roc curve ordinates from the individual split
        cv_roc_curves.append(roc_curve(target[test_idx], model.predict_proba(pool_test)[:, 1], 
                                       drop_intermediate=False))
        
        # save feature importances from the individual split
        feature_importances.append(model.feature_importances_)
        
        # save model itself
        model.save_model(f"../results/models/CGB_{filepath[8:-4]}_Split{i+1}")
        
        # save indexes
        indexes[f"Split{i+1}"] = {"train": train_idx, "test": test_idx}
    
    cv_scores = pd.DataFrame(np.array(cv_scores),
                             index=["Split1", "Split2", "Split3", "Split4", "Split5"],
                             columns=["AUC"])
    
    cv_roc_curves = {cv_scores.index[i]: {"FPR": cv_roc_curves[i][0], 
                                          "TPR": cv_roc_curves[i][1]} for i in range(len(cv_roc_curves))}
    
    feature_importances = pd.DataFrame(np.array(feature_importances), 
                                       index=["Split1", "Split2", "Split3", "Split4", "Split5"], 
                                       columns=features.columns)
    
    return cv_scores, cv_roc_curves, feature_importances, indexes

In [4]:
%%time

# loop over individual Time Intervals
for i in range(1, 5):
    
    # obtain scores and feature importances using cross-validation
    cv, rc, fi, ids = model_me(f"../data/TimeInterval{i}.csv")
    
    cv.to_csv(f"../results/CGB_TimeInterval{i}_AUC.csv")
    fi.to_csv(f"../results/CGB_TimeInterval{i}_FI.csv")
    
    np.save(f"../results/CGB_TimeInterval{i}_RC.npy", rc)
    np.save(f"../results/models/Split_indexes.npy", ids)
    
    print(cv.mean())
    print(fi.mean())

AUC    0.686198
dtype: float64
K_Numeric          7.652976
Min_Numeric        8.888872
C_Numeric          7.636281
S_Numeric          9.014229
O_Numeric         10.463883
T_Numeric          6.318961
M_Numeric          5.989049
R_Numeric          7.541339
Re_Numeric         6.309838
MinD_Numeric      21.185563
Phylum_Numeric     8.999009
dtype: float64
AUC    0.761128
dtype: float64
K_Numeric          6.381792
Min_Numeric        8.350518
C_Numeric         11.930064
S_Numeric          9.189230
O_Numeric          7.614511
T_Numeric          8.948648
M_Numeric          5.515479
R_Numeric         10.325281
Re_Numeric         5.379933
MinD_Numeric      19.851235
Phylum_Numeric     6.513308
dtype: float64
AUC    0.757319
dtype: float64
K_Numeric         13.120803
Min_Numeric       13.218029
C_Numeric          7.399258
S_Numeric         10.659577
O_Numeric          9.606583
T_Numeric          6.407696
M_Numeric         10.536500
R_Numeric          5.982243
Re_Numeric         8.257403
MinD_Nume